# Building a NN and training it using the CIFAR10 dataset.

## Importing libraries

In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_gpu as tf
from tensorflow import keras
import os
from sklearn.model_selection import RandomizedSearchCV

ModuleNotFoundError: No module named 'tensorflow_gpu'

In [83]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [80]:
!pip3 list

Package                  Version      
------------------------ -------------
absl-py                  0.9.0        
astunparse               1.6.3        
attrs                    19.3.0       
Automat                  0.8.0        
backcall                 0.1.0        
beautifulsoup4           4.9.1        
bleach                   3.1.5        
blinker                  1.4          
bs4                      0.0.1        
cachetools               4.1.0        
certifi                  2019.11.28   
chardet                  3.0.4        
Click                    7.0          
cloud-init               20.1         
colorama                 0.4.3        
command-not-found        0.3          
configobj                5.0.6        
constantly               15.1.0       
cryptography             2.8          
cvxopt                   1.2.5        
cycler                   0.10.0       
dbus-python              1.2.16       
decorator                4.4.2        
defusedxml               

## Loading dataset

In [2]:
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [3]:
x_train_full.shape

(50000, 32, 32, 3)

In [4]:
x_test.shape

(10000, 32, 32, 3)

In [5]:
x_valid, x_train = x_train_full[:5000], x_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

## Building the model

In [6]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [7]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape = [32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, activation = keras.activations.relu, kernel_initializer = keras.initializers.he_normal()))
model.add(keras.layers.Dense(10, activation = keras.activations.softmax))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               307300    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               1

- Now we need to find an optimal learning rate. To do this we will use a Nadam optimizer and train the model for 10 epochs with different learning rates.
- We will be setting up a tensorboard environment using a tensorboard callback and check the performance of the model.

In [9]:
learning_rate = 0.00025

In [10]:
run_logdir = os.path.join(os.curdir, 'CIFAR10_logs', 'optimal_lr_model', 'lr = {}'.format(learning_rate))
tb_callback = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('CIFAR10_model.h5', save_best_only = True)

In [11]:
model.compile(loss = keras.losses.sparse_categorical_crossentropy, optimizer = keras.optimizers.Nadam(learning_rate = learning_rate), metrics = 'accuracy')

In [12]:
model.fit(x_train, y_train, epochs = 10, validation_data = (x_valid, y_valid), callbacks = [tb_callback])

Epoch 1/10
   1/1407 [..............................] - ETA: 0s - loss: 165.7273 - accuracy: 0.0625WARNING:tensorflow:From /home/gokul/.local/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1407/1407 [==============================] - 8s 6ms/step - loss: 2.7819 - accuracy: 0.1722 - val_loss: 2.0820 - val_accuracy: 0.2038
Epoch 2/10
1407/1407 [==============================] - 7s 5ms/step - loss: 1.9937 - accuracy: 0.2514 - val_loss: 2.1159 - val_accuracy: 0.2318
Epoch 3/10
1407/1407 [==============================] - 9s 6ms/step - loss: 1.9164 - accuracy: 0.2812 - val_loss: 1.9130 - val_accuracy: 0.2824
Epoch 4/10
1407/1407 [==============================] - 10s 7ms/step - loss: 1.8650 - accuracy: 0.3061 - val_loss: 1.8742 - val_accuracy: 0.3116
Epoch 5/10
1407/1407 [===========================

- From multiple trial and error values for learning rate, it was found that the model performed the best at a learning rate of 0.00025.

In [13]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [14]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape = [32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, activation = keras.activations.relu, kernel_initializer = keras.initializers.he_normal()))
model.add(keras.layers.Dense(10, activation = keras.activations.softmax))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               307300    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               1

In [16]:
optimal_lr = 0.00025

In [17]:
model.compile(loss = keras.losses.sparse_categorical_crossentropy, optimizer = keras.optimizers.Nadam(learning_rate = optimal_lr), metrics = 'accuracy')

In [18]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('CIFAR10_model', save_best_only=True, save_weights_only = True)
run_logdir = os.path.join(os.curdir, 'CIFAR10_logs', 'optimal_lr_model')
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [19]:
history = model.fit(x_train, y_train, epochs = 100, validation_data = (x_valid, y_valid), callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb])

Epoch 1/100
1407/1407 [==============================] - 9s 7ms/step - loss: 2.7819 - accuracy: 0.1722 - val_loss: 2.0820 - val_accuracy: 0.2038
Epoch 2/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.9937 - accuracy: 0.2514 - val_loss: 2.1159 - val_accuracy: 0.2318
Epoch 3/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.9164 - accuracy: 0.2812 - val_loss: 1.9130 - val_accuracy: 0.2824
Epoch 4/100
1407/1407 [==============================] - 8s 5ms/step - loss: 1.8650 - accuracy: 0.3061 - val_loss: 1.8742 - val_accuracy: 0.3116
Epoch 5/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.8361 - accuracy: 0.3218 - val_loss: 1.9204 - val_accuracy: 0.2920
Epoch 6/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.7965 - accuracy: 0.3409 - val_loss: 1.8150 - val_accuracy: 0.3338
Epoch 7/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.7458 - accuracy: 0.3650 - val_loss: 1.7359 - val_ac

In [20]:
model.evaluate(x_valid, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.5790 - accuracy: 0.4686


[1.5790321826934814, 0.46860000491142273]

In [21]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.5602 - accuracy: 0.4689


[1.5602147579193115, 0.46889999508857727]

In [22]:
model.load_weights('CIFAR10_model')

In [23]:
model.evaluate(x_valid, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.5258 - accuracy: 0.4720


[1.5257840156555176, 0.47200000286102295]

In [24]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.5075 - accuracy: 0.4742


[1.5074900388717651, 0.4742000102996826]

- We can see that the model with the least validation_loss gives us am accuracy of 47% on the validation and test data.

## Adding batch normalization

- We will be adding the batvh normalization layers in the net and check its performance variations.
- Again as we changed the model architecture, we need to find the optimal learning rate by trial and error.

In [175]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [176]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape = [32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer = keras.initializers.he_normal()))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('elu'))
model.add(keras.layers.Dense(10, activation = keras.activations.softmax))

In [177]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 3072)              12288     
_________________________________________________________________
dense (Dense)                (None, 100)               307300    
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               4

In [178]:
learning_rate = 0.0015

In [179]:
model.compile(loss = keras.losses.sparse_categorical_crossentropy, optimizer = keras.optimizers.Nadam(lr = learning_rate), metrics = 'accuracy')

In [180]:
run_logdir = os.path.join(os.curdir, 'CIFAR10_logs', 'BN', 'lr = {}'.format(learning_rate))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [181]:
history = model.fit(x_train, y_train, epochs = 20, validation_data = (x_valid, y_valid), callbacks = [tensorboard_cb])

Epoch 1/20
1407/1407 [==============================] - 18s 13ms/step - loss: 1.8514 - accuracy: 0.3351 - val_loss: 1.7488 - val_accuracy: 0.3762
Epoch 2/20
1407/1407 [==============================] - 17s 12ms/step - loss: 1.7044 - accuracy: 0.3926 - val_loss: 1.6794 - val_accuracy: 0.3950
Epoch 3/20
1407/1407 [==============================] - 18s 13ms/step - loss: 1.6400 - accuracy: 0.4171 - val_loss: 1.6474 - val_accuracy: 0.4116
Epoch 4/20
1407/1407 [==============================] - 16s 11ms/step - loss: 1.5878 - accuracy: 0.4350 - val_loss: 1.6109 - val_accuracy: 0.4320
Epoch 5/20
1407/1407 [==============================] - 15s 11ms/step - loss: 1.5416 - accuracy: 0.4529 - val_loss: 1.5223 - val_accuracy: 0.4540
Epoch 6/20
1407/1407 [==============================] - 15s 11ms/step - loss: 1.5039 - accuracy: 0.4670 - val_loss: 1.4697 - val_accuracy: 0.4698
Epoch 7/20
1407/1407 [==============================] - 15s 10ms/step - loss: 1.4666 - accuracy: 0.4794 - val_loss: 1.4893 -

- After using different learning rates, it was found that the model having lr = 0.001 gives the best performance.

In [240]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [241]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape = [32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer = keras.initializers.he_normal()))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('elu'))
model.add(keras.layers.Dense(10, activation = keras.activations.softmax))

In [242]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 3072)              12288     
_________________________________________________________________
dense (Dense)                (None, 100)               307300    
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               4

In [243]:
optimal_lr = 0.001

In [244]:
model.compile(loss = keras.losses.sparse_categorical_crossentropy, optimizer = keras.optimizers.Nadam(learning_rate = optimal_lr), metrics = ['accuracy'])

In [245]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('CIFAR10_BN_model', save_best_only = True, save_weights_only = True)
run_logdir = os.path.join(os.curdir, 'CIFAR10_logs', 'BN', 'optimal_lr_model', 'lr = {}'.format(optimal_lr))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [246]:
history = model.fit(x_train, y_train, epochs = 100, validation_data = (x_valid, y_valid), callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb])

Epoch 1/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.8374 - accuracy: 0.3410 - val_loss: 1.6944 - val_accuracy: 0.4008
Epoch 2/100
1407/1407 [==============================] - 15s 11ms/step - loss: 1.6859 - accuracy: 0.3998 - val_loss: 1.6820 - val_accuracy: 0.3850
Epoch 3/100
1407/1407 [==============================] - 15s 11ms/step - loss: 1.6192 - accuracy: 0.4231 - val_loss: 1.6028 - val_accuracy: 0.4250
Epoch 4/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.5717 - accuracy: 0.4369 - val_loss: 1.5617 - val_accuracy: 0.4480
Epoch 5/100
1407/1407 [==============================] - 15s 11ms/step - loss: 1.5286 - accuracy: 0.4608 - val_loss: 1.4750 - val_accuracy: 0.4708
Epoch 6/100
1407/1407 [==============================] - 15s 11ms/step - loss: 1.4899 - accuracy: 0.4730 - val_loss: 1.4929 - val_accuracy: 0.4742
Epoch 7/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.4539 - accuracy: 0.4841 - val_loss: 1

In [247]:
model.evaluate(x_valid, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.3898 - accuracy: 0.5472


[1.389768123626709, 0.5472000241279602]

In [248]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.3966 - accuracy: 0.5371


[1.3966196775436401, 0.5371000170707703]

In [249]:
model.load_weights('CIFAR10_BN_model')

In [250]:
model.evaluate(x_valid, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.3185 - accuracy: 0.5530


[1.3185102939605713, 0.5529999732971191]

In [251]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.3248 - accuracy: 0.5398


[1.3248482942581177, 0.5397999882698059]

- We can see that the model reached the lowest val_loss at 23 epochs when we added BN to the net.
- Adding BN enabled us to use much larger lr which reduced the total training time.
- The metrics of the model are also better with the validation and test accuracies being 55% and 53% respectively.
- The only drawback is that the time taken to complete each epoch increased from 10s to 16s, but on overall the total training time is decreased.

## Replacing BN with SELU

- We need to standardize the inputs in order to use the SELU activation func.

In [6]:
x_mean = x_train.mean(axis = 0, keepdims = True)
x_std = x_train.std(axis = 0, keepdims = True)

In [7]:
x_train_scaled = (x_train - x_mean) / x_std
x_valid_scaled = (x_valid - x_mean) / x_std
x_test_scaled = (x_test - x_mean) / x_std

In [342]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [343]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape = [32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, activation = keras.activations.selu, kernel_initializer = keras.initializers.lecun_normal()))
model.add(keras.layers.Dense(10, activation = keras.activations.softmax))

In [344]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               307300    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               1

In [345]:
learning_rate = 0.0008

In [346]:
model.compile(loss = keras.losses.sparse_categorical_crossentropy, optimizer = keras.optimizers.Nadam(learning_rate), metrics = ['accuracy'])

In [347]:
run_logdir = os.path.join(os.curdir, 'CIFAR10_logs', 'SELU', 'lr = {}'.format(learning_rate))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [348]:
history = model.fit(x_train_scaled, y_train, epochs = 20, validation_data = (x_valid_scaled, y_valid), callbacks = [tensorboard_cb])

Epoch 1/20
1407/1407 [==============================] - 9s 7ms/step - loss: 1.9313 - accuracy: 0.3097 - val_loss: 1.8588 - val_accuracy: 0.3136
Epoch 2/20
1407/1407 [==============================] - 9s 6ms/step - loss: 1.7200 - accuracy: 0.3890 - val_loss: 1.7384 - val_accuracy: 0.3778
Epoch 3/20
1407/1407 [==============================] - 9s 6ms/step - loss: 1.6249 - accuracy: 0.4275 - val_loss: 1.7290 - val_accuracy: 0.3988
Epoch 4/20
1407/1407 [==============================] - 8s 6ms/step - loss: 1.5634 - accuracy: 0.4478 - val_loss: 1.6357 - val_accuracy: 0.4336
Epoch 5/20
1407/1407 [==============================] - 9s 6ms/step - loss: 1.5074 - accuracy: 0.4691 - val_loss: 1.5887 - val_accuracy: 0.4392
Epoch 6/20
1407/1407 [==============================] - 9s 6ms/step - loss: 1.4673 - accuracy: 0.4866 - val_loss: 1.5583 - val_accuracy: 0.4644
Epoch 7/20
1407/1407 [==============================] - 8s 6ms/step - loss: 1.4314 - accuracy: 0.5005 - val_loss: 1.5709 - val_accuracy:

- After changing the architecture we could find that the model performs at its best at lr = 0.0004.

In [363]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [364]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape = [32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, activation = keras.activations.selu, kernel_initializer = keras.initializers.lecun_normal()))
model.add(keras.layers.Dense(10, activation = keras.activations.softmax))

In [365]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               307300    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               1

In [366]:
optimal_lr = 0.0004

In [367]:
model.compile(loss = keras.losses.sparse_categorical_crossentropy, optimizer = keras.optimizers.Nadam(optimal_lr), metrics = ['accuracy'])

In [368]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('CIFAR10_SELU_model', save_best_only = True, save_weights_only = True)
run_logdir = os.path.join(os.curdir, 'CIFAR10_logs', 'SELU', 'optimal_model', 'lr = {}'.format(optimal_lr))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [369]:
history = model.fit(x_train_scaled, y_train, epochs = 100, validation_data = (x_valid_scaled, y_valid), callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb])

Epoch 1/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.8783 - accuracy: 0.3294 - val_loss: 1.7889 - val_accuracy: 0.3518
Epoch 2/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.6694 - accuracy: 0.4085 - val_loss: 1.7444 - val_accuracy: 0.3730
Epoch 3/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.5651 - accuracy: 0.4500 - val_loss: 1.5895 - val_accuracy: 0.4310
Epoch 4/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.4906 - accuracy: 0.4757 - val_loss: 1.5508 - val_accuracy: 0.4604
Epoch 5/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.4261 - accuracy: 0.4994 - val_loss: 1.5326 - val_accuracy: 0.4566
Epoch 6/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.3760 - accuracy: 0.5150 - val_loss: 1.4838 - val_accuracy: 0.4800
Epoch 7/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.3262 - accuracy: 0.5373 - val_loss: 1.4950 - val_

In [371]:
model.evaluate(x_valid_scaled, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.6528 - accuracy: 0.5068


[1.652758240699768, 0.5067999958992004]

In [372]:
model.evaluate(x_test_scaled, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.6576 - accuracy: 0.5017


[1.657639980316162, 0.5016999840736389]

In [373]:
model.load_weights('CIFAR10_SELU_model')

In [374]:
model.evaluate(x_valid_scaled, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.4441 - accuracy: 0.5000


[1.4440743923187256, 0.5]

In [376]:
model.evaluate(x_test_scaled, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.4459 - accuracy: 0.4979


[1.445914626121521, 0.49790000915527344]

- It is the fastest model by far as the best model in terms of val_error was achieved in 8 epochs with 10s for each epoch.
- But the drawback is the model performance which even though is better than the raw model is less than that of the model with BN layers.

## Using AlphaDropouts for regularization

In [57]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [58]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape = [32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, activation = keras.activations.selu, kernel_initializer = keras.initializers.lecun_normal()))
model.add(keras.layers.AlphaDropout(rate = 0.1))
model.add(keras.layers.Dense(10, activation = keras.activations.softmax))

In [59]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
alpha_dropout (AlphaDropout) (None, 3072)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               307300    
_________________________________________________________________
alpha_dropout_1 (AlphaDropou (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               1

In [60]:
model.compile(loss = keras.losses.sparse_categorical_crossentropy, optimizer = keras.optimizers.Nadam(), metrics = ['accuracy'])

In [61]:
run_logdir = os.path.join(os.curdir, 'CIFAR10_logs', 'Dropout', 'Div_3_dropout')
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [62]:
history = model.fit(x_train_scaled, y_train, epochs = 20, validation_data = (x_valid_scaled, y_valid), callbacks = [tensorboard_cb])

Epoch 1/20
1407/1407 [==============================] - 11s 8ms/step - loss: 2.0495 - accuracy: 0.2426 - val_loss: 2.1186 - val_accuracy: 0.3018
Epoch 2/20
1407/1407 [==============================] - 9s 7ms/step - loss: 1.8852 - accuracy: 0.3040 - val_loss: 2.1627 - val_accuracy: 0.3038
Epoch 3/20
1407/1407 [==============================] - 10s 7ms/step - loss: 1.8247 - accuracy: 0.3313 - val_loss: 2.4859 - val_accuracy: 0.3388
Epoch 4/20
1407/1407 [==============================] - 10s 7ms/step - loss: 1.7897 - accuracy: 0.3410 - val_loss: 2.2891 - val_accuracy: 0.3356
Epoch 5/20
1407/1407 [==============================] - 11s 8ms/step - loss: 1.7680 - accuracy: 0.3544 - val_loss: 2.8053 - val_accuracy: 0.3532
Epoch 6/20
1407/1407 [==============================] - 11s 7ms/step - loss: 1.7588 - accuracy: 0.3532 - val_loss: 2.3630 - val_accuracy: 0.3522
Epoch 7/20
1407/1407 [==============================] - 11s 8ms/step - loss: 1.7349 - accuracy: 0.3613 - val_loss: 2.2733 - val_acc

- After training the model we could see that the model performs the best when dropout is added after the last hidden layer.
- Now we need to decide the dropout rate as well as the learning_rate for which we will be using Randomized Search CV.

In [70]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [71]:
def build_model(dropout_rate = 0.1, learning_rate = 1e-3):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape = [32, 32, 3]))
    for _ in range(20):
        model.add(keras.layers.Dense(100, activation = keras.activations.selu, kernel_initializer = keras.initializers.lecun_normal()))
    model.add(keras.layers.AlphaDropout(rate = dropout_rate))
    model.add(keras.layers.Dense(10, activation = keras.activations.softmax))
    model.compile(loss = keras.losses.sparse_categorical_crossentropy, optimizer = keras.optimizers.Nadam(lr = learning_rate), metrics = ['accuracy'])
    return model

In [72]:
classifier = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [73]:
params = {'dropout_rate' : [0.01, 0.03, 0.1, 0.3], 'learning_rate' : [0.00001, 0.00003, 0.0001, 0.0003, 0.001, 0.003, 0.01]}

In [74]:
rand_search = RandomizedSearchCV(estimator = classifier, param_distributions = params, n_iter = 10)

In [75]:
rand_search.fit(x_train, y_train, epochs = 100, validation_data = (x_valid, y_valid), callbacks = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True))

Epoch 1/100
1125/1125 [==============================] - 7s 6ms/step - loss: 2.0442 - accuracy: 0.2525 - val_loss: 1.8363 - val_accuracy: 0.3266
Epoch 2/100
1125/1125 [==============================] - 6s 6ms/step - loss: 1.8271 - accuracy: 0.3339 - val_loss: 1.7315 - val_accuracy: 0.3678
Epoch 3/100
1125/1125 [==============================] - 6s 5ms/step - loss: 1.7437 - accuracy: 0.3670 - val_loss: 1.7194 - val_accuracy: 0.3822
Epoch 4/100
1125/1125 [==============================] - 7s 6ms/step - loss: 1.6883 - accuracy: 0.3923 - val_loss: 1.6571 - val_accuracy: 0.4010
Epoch 5/100
1125/1125 [==============================] - 6s 6ms/step - loss: 1.6443 - accuracy: 0.4077 - val_loss: 1.6256 - val_accuracy: 0.4118
Epoch 6/100
1125/1125 [==============================] - 6s 6ms/step - loss: 1.6014 - accuracy: 0.4276 - val_loss: 1.6270 - val_accuracy: 0.4134
Epoch 7/100
1125/1125 [==============================] - 6s 6ms/step - loss: 1.5743 - accuracy: 0.4348 - val_loss: 1.5981 - val_ac

RandomizedSearchCV(estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f1b35a9aca0>,
                   param_distributions={'dropout_rate': [0.01, 0.03, 0.1, 0.3],
                                        'learning_rate': [1e-05, 3e-05, 0.0001,
                                                          0.0003, 0.001, 0.003,
                                                          0.01]})

In [76]:
rand_search.best_params_

{'learning_rate': 3e-05, 'dropout_rate': 0.03}

In [77]:
rand_search.best_score_

0.46608888506889345